# Big Project: Fantasy Premier League (FPL) Points Predictor
## Data Rules Everything Around Me (DREAM) TEAM - Fall 2024 - CME 538
## Feras Abdulla - Maha Fakhroo - Syed Shahid Hossaini - Eric Guan
-----------------------------------------------------------------------------------------------------------------------------------------------
# Exploratory Data Analysis (EDA) and Visualizations
-----------------------------------------------------------------------------------------------------------------------------------------------


Below, the different columns available in the database are listed and explained:

1) `'name'`: Name of player
2) `'position'`: Position on the pitch (Goalkeeper, Defender, Midfielder, Forward)
3) `'team'`: Premier League team with which the player is affiliated
4) `'xP'`: Expected points for a the player in the given fixture
5) `'assists'`: Actual number of assists
6) `'bonus'`: Actual number of bonus points awarded
7) `'bps'`: Stands for 'Bonus Points System', a raw score based on performance metrics like goals, assists, clean sheets, saves, tackles, and other contributions that is used to rank players and determine `'bonus'` scores.
8) `'clean_sheets'`: Boolean column identifying whether the player earned points for a clean sheet (i.e., his team conceded zero goals while he was on the pitch).
9) `'creativity'`: A measure of a player’s potential to create scoring opportunities (passes, crosses, etc.).
10) `'element'`: A unique ID for the player in the FPL system.
11) `'fixture'`: The ID of the match the player participated in.
12) `'goals_conceded'`: The number of goals the player's team conceded while they were on the pitch.
13) `'goals_scored'`: The number of goals scored by the player.
14) `'Influence_Creativity_Threat_Index'`: A combined metric summarizing the player's influence, creativity, and threat.
15) `'influence'`: A measure of a player’s impact on a match (defensive and offensive contributions).
16) `'kickoff_time'`: The start time of the match.
17) `'minutes'`: The number of minutes the player was on the pitch during the match.
18) `'opponent_team'`: The ID of the opposing team in the fixture.
19) `'own_goals'`: The number of own goals scored by the player.
20) `'penalties_missed'`: The number of penalty kicks missed by the player.
21) `'penalties_saved'`: The number of penalty kicks saved by the player (goalkeepers only).
22) `'red_cards'`: The number of red cards received by the player.
23) `'round'`: The fantasy round number of the match.
24) `'saves'`: The number of saves made by the player (goalkeepers only).
25) `'selected'`: The number of FPL managers who selected the player for their teams in this round.
26) `'team_a_score'`: The number of goals scored by the away team in the match.
27) `'team_h_score'`: The number of goals scored by the home team in the match.
28) `'threat'`: A measure of a player’s likelihood of scoring goals based on their attacking actions.
29) `'total_points'`: The total FPL points earned by the player in the match. `This column will be our label`.
30) `'transfers_balance'`: The net number of transfers for the player (transfers in minus transfers out).
31) `'transfers_in'`: The number of FPL teams that transferred the player in before this match.
32) `'transfers_out'`: The number of FPL teams that transferred the player out before this match.
33) `'value'`: The player’s price in FPL (in millions GBP).
34) `'was_home'`: A boolean indicating if the player's team was playing at home (True/1 = home, False/0 = away).
35) `'yellow_cards'`: The number of yellow cards received by the player.
36) `'GW'`: The specific gameweek for the match.
37) `'expected_goals'`: A metric predicting the likelihood of the player scoring based on their chances.
38) `'expected_assists'`: A metric predicting the likelihood of the player assisting a goal.
39) `'expected_goal_involvements'`: The sum of `'expected_goals'` and `'expected_assists'`, representing the player’s total expected goal contributions.

### Import Packages
Imports essential Python libraries and machine learning tools for data analysis, visualization, and model evaluation, as well as functions for splitting data into training and testing sets. These are typically used in machine learning projects to build and assess predictive models.

In [179]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import ast
import unicodedata

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

## Import Data
Let's start by importing the `'master.csv'` file into a dataframe.

In [180]:
master = pd.read_csv('../master.csv')
master.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,expected_goals,expected_assists,expected_goal_involvements
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,0,0,0,55,True,0,1,0.392763,0.000000,0.392763
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,0,0,0,50,True,0,1,0.000000,0.000000,0.000000
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,0,0,0,50,True,0,1,NaN,NaN,NaN
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0,0,50,True,0,1,0.000000,0.000000,0.000000
4,Abdoulaye DoucourA©,MID,Everton,1.3,0,0,20,1,44.6,512,...,0,0,0,55,False,0,1,0.000000,0.205708,0.205708


In [181]:
master.shape

(111920, 39)

In [182]:
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111920 entries, 0 to 111919
Data columns (total 39 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   name                               111920 non-null  object 
 1   position                           111920 non-null  object 
 2   team                               111920 non-null  object 
 3   xP                                 111920 non-null  float64
 4   assists                            111920 non-null  int64  
 5   bonus                              111920 non-null  int64  
 6   bps                                111920 non-null  int64  
 7   clean_sheets                       111920 non-null  int64  
 8   creativity                         111920 non-null  float64
 9   element                            111920 non-null  int64  
 10  fixture                            111920 non-null  int64  
 11  goals_conceded                     1119

## Data Cleaning

First, let's explore how many missing values exist in the master dataframe:

In [183]:
# Determine missing values for each column
missing_values = master.isnull().sum()

# Create a df of missing values
missing_df = pd.DataFrame({'Missing Values': missing_values})

missing_df

,Missing Values
name,0
position,0
team,0
xP,0
assists,0
bonus,0
bps,0
clean_sheets,0
creativity,0
element,0


So, we see that our efforts to impute/drop missing values will have to focus on three main features: `'expected_goals'`, `'expected_assists'`, and `'expected_goal_involvements'`. Let's go ahead and drop the NaN values, because our large data set will remain substantial to explore and train an ML model even after dropping the missing values.

In [184]:
master = master.dropna()
master.shape

(79272, 39)

Let's also examine the columns that contain categorical data, including how many unique values each contains and what those unique values are. 

In [185]:
# Define categorical columns
categorical_columns = ['position', 'team']

# Calculate how many unique values there are for each categorical column
unique_vals = master[categorical_columns].nunique()
print("Unique values in categorical features:")
print(unique_vals)

# Print different categorical values
for i in categorical_columns:
    print(f"\nUnique values in '{i}':")
    print(master[i].unique())

Unique values in categorical features:
position     5
team        27
dtype: int64

Unique values in 'position':
['FWD' 'DEF' 'GK' 'MID' 'GKP']

Unique values in 'team':
['Brighton' 'West Ham' 'Sheffield Utd' 'Everton' 'Fulham' 'Wolves'
 'Arsenal' 'Southampton' 'Newcastle' 'Chelsea' 'Liverpool'
 'Crystal Palace' 'Leicester' 'Spurs' 'West Brom' 'Leeds' 'Man Utd'
 'Burnley' 'Man City' 'Aston Villa' 'Watford' 'Norwich' 'Brentford'
 'Bournemouth' "Nott'm Forest" 'Luton' 'Ipswich']


As we can see, there are 5 unique player positions: `'FWD'` = Forward, `'DEF'` = Defender, `'MID'` = Midfielder, and `'GK/GKP'` = Goalkeeper. Both GK and GKP refer to the same position but are written differently due to different syntax formats across different seasons, so we will have to align them into one value: `'GK'`. We will also change the abbreviated name of Nottingham Forest in the team column, `'Nott'm Forest'`, to the full name: `'Nottingham Forest'`.

In [186]:
master['position'] = master['position'].replace('GKP', 'GK') # GKP --> GK
master['team'] = master['team'].replace("Nott'm Forest", "Nottingham Forest") # Nott'm Forest --> Nottingham Forest

# Calculate how many unique values there are for each categorical column
unique_vals = master[categorical_columns].nunique()
print("Unique values in categorical features:")
print(unique_vals)

# Print different categorical values
for i in categorical_columns:
    print(f"\nUnique values in '{i}':")
    print(master[i].unique())

Unique values in categorical features:
position     4
team        27
dtype: int64

Unique values in 'position':
['FWD' 'DEF' 'GK' 'MID']

Unique values in 'team':
['Brighton' 'West Ham' 'Sheffield Utd' 'Everton' 'Fulham' 'Wolves'
 'Arsenal' 'Southampton' 'Newcastle' 'Chelsea' 'Liverpool'
 'Crystal Palace' 'Leicester' 'Spurs' 'West Brom' 'Leeds' 'Man Utd'
 'Burnley' 'Man City' 'Aston Villa' 'Watford' 'Norwich' 'Brentford'
 'Bournemouth' 'Nottingham Forest' 'Luton' 'Ipswich']


Let's also examine the data for the presence of duplicate rows.

In [187]:
print(f"Number of duplicate rows in the master data set: {master.duplicated().sum()}")

Number of duplicate rows in the master data set: 0


Nice, no duplicates!


Now, we should filter the `'name'` column to remove any symbols/non-alphabetical characters, to decode the text into normal alphabet format.

In [188]:
def remove_accents(df):
    ''' Replace recurring symbols with their accent-less alphanumeric counterparts '''
    df['name'] = df['name'].str.replace('A©', 'e', regex=False)
    df['name'] = df['name'].str.replace('A§', 'c', regex=False)
    df['name'] = df['name'].str.replace('A­', 'i', regex=False)
    df['name'] = df['name'].str.replace('A3', 'o', regex=False)
    df['name'] = df['name'].str.replace('A¶', 'o', regex=False)
    df['name'] = df['name'].str.replace('A1⁄4', 'u', regex=False)
    df['name'] = df['name'].str.replace('A¤', 'a', regex=False)
    df['name'] = df['name'].str.replace('A«', 'e', regex=False)
    df['name'] = df['name'].str.replace('A£', 'a', regex=False)

    ''' Deal with specific outlier names '''
    df['name'] = df['name'].str.replace('FernA¡ndez', 'Fernandez', regex=False)
    df['name'] = df['name'].str.replace('Marek RodA¡k', 'Marek Rodak', regex=False)
    df['name'] = df['name'].str.replace('GroA', 'Groß', regex=False)
    df['name'] = df['name'].str.replace('Davinson SA¡nchez', 'Davinson Sanchez', regex=False)
    df['name'] = df['name'].str.replace('Cengiz Ander', 'Cengiz Under', regex=False)
    df['name'] = df['name'].str.replace('FabiA¡n Balbuena', 'Fabian Balbuena', regex=False)
    df['name'] = df['name'].str.replace('Robert SA¡nchez', 'Robert Sanchez', regex=False)
    df['name'] = df['name'].str.replace('SaAol Aiguez', 'Saul Niguez', regex=False)
    df['name'] = df['name'].str.replace('Alvaro', 'Alvaro', regex=False)
    df['name'] = df['name'].str.replace('Kanté', 'Kante', regex=False)

    return df

master = remove_accents(master)


Now let us convert the `'kickoff_time'` column into datetime format, extract the `'Hour'`, `'DayOfWeek'`, `'Month'`, `'Weekend'`, and `'WeekOfYear'` elements, and store them into new columns in the `'master'` df.

In [189]:
master['kickoff_time'] = pd.to_datetime(master['kickoff_time'])

master['Hour'] = master['kickoff_time'].dt.hour # Extract hour
master['DayOfWeek'] = master['kickoff_time'].dt.dayofweek # Extract day of week (Monday = 0 to Sunday = 6)
master['Weekend'] = master['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0) # Determine if weekend (1 if yes, 0 if no)
master['WeekOfYear'] = master['kickoff_time'].dt.isocalendar().week # Extract week of year
master['Month'] = master['kickoff_time'].dt.month # Extract month
master['Year'] = master['kickoff_time'].dt.year # Extract year

master.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,GW,expected_goals,expected_assists,expected_goal_involvements,Hour,DayOfWeek,Weekend,WeekOfYear,Month,Year
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,1,0.392763,0.000000,0.392763,19,0,0,38,9,2020
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,1,0.000000,0.000000,0.000000,19,5,1,37,9,2020
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,1,0.000000,0.000000,0.000000,17,0,0,38,9,2020
4,Abdoulaye Doucoure,MID,Everton,1.3,0,0,20,1,44.6,512,...,1,0.000000,0.205708,0.205708,15,6,1,37,9,2020
5,Aboubakar Kamara,MID,Fulham,0.4,0,0,-2,0,1.8,190,...,1,0.020672,0.000000,0.020672,11,5,1,37,9,2020


Now, to encode the categorical features. We will one-hot encode the `'position'` column, since there are only 4 unique values. However, we will label-encode the `'team'` column and create a new column: `'team_label'`. We do this because we have 27 unique teams in the database, and one-hot encoding `'team'` would increase dimensionality substantially. Furthermore, our ML model will be a Random Forest, which does not infer ordinality, so label encoding will not be an issue.

In [190]:
# One-hot encode the 'position' column while retaining the original column
position_dummies = pd.get_dummies(master['position'], prefix='position')
master = pd.concat([master, position_dummies], axis=1)

# Label encode the 'team' column while retaining the original column
le = LabelEncoder()
master['team_label'] = le.fit_transform(master['team'])

master.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,DayOfWeek,Weekend,WeekOfYear,Month,Year,position_DEF,position_FWD,position_GK,position_MID,team_label
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,0,0,38,9,2020,False,True,False,False,4
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,5,1,37,9,2020,True,False,False,False,25
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0,38,9,2020,False,False,True,False,20
4,Abdoulaye Doucoure,MID,Everton,1.3,0,0,20,1,44.6,512,...,6,1,37,9,2020,False,False,False,True,8
5,Aboubakar Kamara,MID,Fulham,0.4,0,0,-2,0,1.8,190,...,5,1,37,9,2020,False,False,False,True,9


Now, we will filter out rows where the number of minutes played, `'minutes'`, is zero. The original `'master'` dataframe includes all players in a season, including those who are listed in a team's squad but do not play (those on the bench).  

In [191]:
master = master[master['minutes'] > 0]

print('Length of master dataframe after filtering out players with 0 minutes of play:', master.shape)

Length of master dataframe after filtering out players with 0 minutes of play: (42529, 50)


As we can see, a large portion of the database (36,743 rows or 46.3%) represented players who were not active in a given fixture. By filtering these out, we can focus on players with concrete contributions when creating visualizations and designing our model.

Now, let us examine invalid occurrences of expected goal metrics. If a player's `'goals_scored'` are greater than zero, then that row's `'expected_goals'` cannot be zero. Therefore, we need to examine the dataframe for those conditions and filter them out.

In [192]:
# Filter rows where goals_scored > 0 and expected_goals == 0
invalid_rows = master[(master['goals_scored'] > 0) & (master['expected_goals'] == 0)]

# Count how many times this happens
invalid_count = len(invalid_rows)

print(f"Number of rows where goals_scored > 0 but expected_goals = 0: {invalid_count}")

Number of rows where goals_scored > 0 but expected_goals = 0: 327


In [193]:
# Drop rows where goals_scored > 0 and expected_goals == 0
master = master[~((master['goals_scored'] > 0) & (master['expected_goals'] == 0))]

print('Length of master dataframe after filtering out rows where goals >0 but expected goals = 0:', master.shape)

Length of master dataframe after filtering out rows where goals >0 but expected goals = 0: (42202, 50)


The number of rows where goals were scored but expected goals were zero are relatively few (327) and have now been filtered out.

We should also filter out rows with limited `'minutes'` of gameplay, because they tend to have incomplete or missing data. They may also bias our model and analysis by including late-game strategies (i.e., some managers might substitute a player in at the end of a game where his team is leading to increase defensive posture and maintain their lead). However, in order to avoid favoring early starters, we need to strike a balance in choosing the `'minutes'` threshold.

We will proceed by filtering out players with less than 5 minutes of gameplay.

In [194]:
master = master[master['minutes'] >= 5]

print('Length of master daraframe after filtering out rows with less than 5 minutes of gameplay: ', master.shape)

Length of master daraframe after filtering out rows with less than 5 minutes of gameplay:  (40462, 50)


So, 1740 rows have been filtered out.

Now, we need to check for negative values

In [195]:
for column in master.columns:
    # Check if the column is numeric
    if master[column].dtype in ['int64', 'float64']:
        # Filter rows with negative values
        negatives = master[master[column] < 0]
        if not negatives.empty:
            print(f"Negative values found in column '{column}':")
            print(len(negatives))
            print("\n")

Negative values found in column 'xP':
1590


Negative values found in column 'bps':
1440


Negative values found in column 'total_points':
491


Negative values found in column 'transfers_balance':
20529




`'total_points'`, `'bps'`, and `'transfers_balance'` can have negative values. Players can be penalized for events like own goals, red cards, and goals conceded, so those negatives can and should be retained. In addition, `'transfers_balance'` is a net figure that represents transfers in minus transfers out, so no issue with negatives here either.

However, `'xP'` values are generally non-negative because they are probabilities multiplied by point weights. Negative xPs can indicate errors in data sourcing. Negative `'xP'` will thus be imputed with the average of that player's xP from the gameweeks before and after the negative value. Care will be taken so that there is no jumping between seasons, since the `'master'` dataframe is a concatenation of several seasons. If adjacent xPs for that same player are also negative, it will be replaced with the closest neighbor.

In [200]:
# Replace negative xP values with NaN
master.loc[master['xP'] < 0, 'xP'] = np.nan

# Function to impute xP
def impute_xp(df):
    # Iterate through each player's data
    for name, group in df.groupby('name'):
        # Loop through rows with NaN in xP
        for idx in group[group['xP'].isna()].index:
            current_gw = df.loc[idx, 'GW']

            # Check for previous and next GWs in the same season
            previous_idx = group[
                (group['GW'] < current_gw) & (~group['xP'].isna())
            ].index.max()
            next_idx = group[
                (group['GW'] > current_gw) & (~group['xP'].isna())
            ].index.min()

            if pd.notna(previous_idx) and pd.notna(next_idx):
                # Average of the previous and next valid xP values
                df.loc[idx, 'xP'] = (df.loc[previous_idx, 'xP'] + df.loc[next_idx, 'xP']) / 2
            elif pd.notna(previous_idx):
                # Use the previous valid xP value
                df.loc[idx, 'xP'] = df.loc[previous_idx, 'xP']
            elif pd.notna(next_idx):
                # Use the next valid xP value
                df.loc[idx, 'xP'] = df.loc[next_idx, 'xP']
            else:
                # Fallback: Use the closest available xP value
                neighbor_idx = group[~group['xP'].isna()].index.min()
                if pd.notna(neighbor_idx):
                    df.loc[idx, 'xP'] = df.loc[neighbor_idx, 'xP']
    return df

# Apply the imputation function
master = impute_xp(master)

# Verify the result
print(len(master[master['xP'].isna()]))  # Should be empty if all NaNs are imputed

29


The remaining NaNs are likely due to zero applicable values to impute with, based on our imputation conditions. Therefore, let's go ahead and drop these 29 rows.

In [201]:
print(master.shape)
master = master.dropna()
print(master.shape)

(40462, 50)
(40433, 50)


Now, finally, let's go ahead and save a cleaned master file to CSV format.

In [202]:
master.to_csv('../master_cleaned.csv', index=False)

## Visualizations

First, let's import the cleaned/filtered data into a new dataframe called `'master_cleaned'`

In [203]:
master_cleaned = pd.read_csv('../master_cleaned.csv')
master_cleaned.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,DayOfWeek,Weekend,WeekOfYear,Month,Year,position_DEF,position_FWD,position_GK,position_MID,team_label
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,0,0,38,9,2020,False,True,False,False,4
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,5,1,37,9,2020,True,False,False,False,25
2,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0,0,38,9,2020,False,False,True,False,20
3,Abdoulaye Doucoure,MID,Everton,1.3,0,0,20,1,44.6,512,...,6,1,37,9,2020,False,False,False,True,8
4,Aboubakar Kamara,MID,Fulham,0.4,0,0,-2,0,1.8,190,...,5,1,37,9,2020,False,False,False,True,9
